In [315]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [233]:
from dataclasses import dataclass, field
import typing as t
import itertools as it
import collections as c
import json
from copy import deepcopy
import math
import time
import functools as ft

In [82]:
def default_player():
    return dict(
        ore=0,
        clay=0,
        obsidian=0,
        geode=0,
        rate=dict(
            ore=1,
            clay=0,
            obsidian=0,
            geode=0,
        ),
    )

def str_player(player):
    return str("Player({})".format(", ".join(f"{k}={v}"for k, v in player.items())))

In [348]:
import re

def parse_raw_input(fl):
    bps = {}
    input_lns = open("input.txt").read().splitlines()
    for ln in input_lns:
        bp_id, ore_rbt_ore_cost, cly_rbt_ore_cost, obs_rbt_ore_cost, obs_rbt_clay_cost, geode_rbt_ore_cost, geode_rbt_obs_cost  = (
            int(n)
            for n in re.findall(
            r"Blueprint (\d+): Each ore robot costs (\d+) ore. Each clay robot costs (\d+) ore. Each obsidian robot costs (\d+) ore and (\d+) clay. Each geode robot costs (\d+) ore and (\d+) obsidian.",
            ln,
            )[0]
        )
        bps[bp_id] = {
            "ore_robot": {"ore": ore_rbt_ore_cost},
            "clay_robot": {"ore": cly_rbt_ore_cost},
            "obsidian_robot": {"ore": obs_rbt_ore_cost, "clay": obs_rbt_clay_cost},
            "geode_robot": {"ore": geode_rbt_ore_cost, "obsidian": geode_rbt_obs_cost},
        }
    return bps

In [349]:
input_bps = parse_input("parsed_input.txt")
raw_bps = parse_raw_input("input.txt")

In [350]:
input_bps==raw_bps

False

In [351]:
len(input_bps), len(raw_bps)

(30, 30)

In [353]:
input_bps[2] 

{'ore_robot': {'ore': 4},
 'clay_robot': {'ore': 4},
 'obsidian_robot': {'ore': 4, 'clay': 1},
 'geode_robot': {'ore': 2, 'obsidian': 15}}

In [354]:
raw_bps[2]

{'ore_robot': {'ore': 4},
 'clay_robot': {'ore': 4},
 'obsidian_robot': {'ore': 4, 'clay': 16},
 'geode_robot': {'ore': 2, 'obsidian': 15}}

In [352]:
for i in range(1, 31):
    print(i, input_bps[i] == raw_bps[i])

1 True
2 False
3 False
4 False
5 False
6 False
7 False
8 False
9 False
10 True
11 True
12 False
13 False
14 True
15 False
16 True
17 True
18 False
19 True
20 False
21 False
22 False
23 False
24 False
25 False
26 True
27 False
28 False
29 False
30 False


In [83]:
str_player(default_player())

"Player(ore=0, clay=0, obsidian=0, geode=0, rate={'ore': 1, 'clay': 0, 'obsidian': 0, 'geode': 0})"

### Test data

In [68]:
def parse_input(fl, lns):
    blue_prints = [json.loads(ln) for ln in  open(fl).read().splitlines()]
    out = {}
    for bp in blue_prints:
        bp = deepcopy(bp)
        bp_num = bp['Blueprint']
        del bp['Blueprint']
        out[bp_num] = bp
    return out

In [30]:
test_bps = parse_input("parsed_test.txt")
input_bps = parse_input("parsed_input.txt")
test_bps

{1: {'ore_robot': {'ore': 4},
  'clay_robot': {'ore': 2},
  'obsidian_robot': {'ore': 3, 'clay': 14},
  'geode_robot': {'ore': 2, 'obsidian': 7}},
 2: {'ore_robot': {'ore': 2},
  'clay_robot': {'ore': 3},
  'obsidian_robot': {'ore': 3, 'clay': 8},
  'geode_robot': {'ore': 3, 'obsidian': 12}}}

In [133]:
def next_states(blue_print, player):
    """Yields (plus_time, robot_to_build) 
    """
    opts = []
    for robot, reqs in blue_print.items():
        rbt_nxt_time = 0
        for rsrc, min_val in reqs.items():
            if player[rsrc] >= min_val:
                continue
            if player["rate"][rsrc] == 0:
                rbt_nxt_time = float('inf')
                continue
            rbt_nxt_time = max(
                math.ceil((min_val - player[rsrc])/player["rate"][rsrc]),
                rbt_nxt_time,
            )
        if rbt_nxt_time != float('inf'):
            opts.append((rbt_nxt_time, robot))
    for opt insorted(opts):
        yield opt

In [134]:
p = default_player()
p["rate"]["obsidian"] = 7 
p["rate"]["clay"] = 1 

list(next_states(test_bps[1], player=p))

[(2, 'clay_robot'),
 (2, 'geode_robot'),
 (4, 'ore_robot'),
 (14, 'obsidian_robot')]

In [214]:
bp

{'ore_robot': {'ore': 4},
 'clay_robot': {'ore': 2},
 'obsidian_robot': {'ore': 3, 'clay': 14},
 'geode_robot': {'ore': 2, 'obsidian': 7}}

In [221]:
bp

{'ore_robot': {'ore': 4},
 'clay_robot': {'ore': 2},
 'obsidian_robot': {'ore': 3, 'clay': 14},
 'geode_robot': {'ore': 2, 'obsidian': 7}}

In [220]:
[reqs['ore'] for reqs in bp.values()]

[4, 2, 3, 2]

In [ ]:
def max_geodes_bp(blueprint, max_time=24, toprint=False) -> int:
    player = default_player()
    return max_geodes_rec(
        bp=blueprint,
        player=deepcopy(player),
        cur_time=0,
        max_time=max_time,
        cache={},
        toprint=toprint,
    )


def max_geodes_rec(bp, player, cur_time, max_time, cache, toprint) -> int:
    if cur_time == max_time:
        return player["geode"]
    ckey = _cache_key(cur_time, player)
    if ckey in cache:
        return cache[ckey]
    buildable_bots = list(buildable_robots(bp=bp, player=player))
    _update_player_rsrc_inp(player=player)
    dflt_kwargs = dict(bp=bp, cur_time=cur_time+1, cache=cache, toprint=toprint, max_time=max_time)
    build_opts = [
        max_geodes_rec(
            player=_build_bot_inp(bot=bot, bot_reqs=bp[bot], player=deepcopy(player)),
            **dflt_kwargs,
        )
        for bot in buildable_bots
    ]
    if toprint:
        print(f"Time: {cur_time}, {player}, buildable bots: {set(buildable_robots(bp=bp, player=player))}")
    nobuild_geodes = max_geodes_rec(player=deepcopy(player), **dflt_kwargs)
    build_geodes = max(build_opts) if build_opts else 0
    cache[ckey] = max(build_geodes, nobuild_geodes)
    return cache[ckey]


def _bot_to_rsrc(bot):
    if bot is None:
        return None
    return bot.rstrip("_robot")


def _build_bot_inp(bot, bot_reqs, player):
    for rsrc, rsrc_cnt in bot_reqs.items():
        player[rsrc] -= rsrc_cnt
    player["rate"][_bot_to_rsrc(bot)] += 1
    return player
    

def _update_player_rsrc_inp(player):
    for rsrc, rate in player["rate"].items():
        player[rsrc] += rate


def buildable_robots(bp, player):
    if _is_buildable(bot_reqs=bp['geode_robot'], player=player):
        yield 'geode_robot'
        return
    for robot, reqs in bp.items():
        if not _is_buildable(bot_reqs=reqs, player=player):
            continue
        if robot == 'geode_robot':
            yield robot
            continue
        # maximum required number of rsrcs tht is created by this bot
        # if we already are producing max num, there's no point building
        # more of such bots
        bot_rsrc = _bot_to_rsrc(robot)
        max_reqmt = max_rsrc_reqmt(bp=bp, bot_rsrc=bot_rsrc)
        if player["rate"][bot_rsrc] < max_reqmt:
            yield robot

def max_rsrc_reqmt(bp, bot_rsrc):
    return max(reqs.get(bot_rsrc, 0) for reqs in bp.values())


def _is_buildable(bot_reqs, player):
    for rsrc, min_val in bot_reqs.items():
        if player[rsrc] < min_val:
            return False
    else:
        return True


def _cache_key(cur_time, player):
    rates = player["rate"]
    return (
        f"{cur_time}, {player['ore']}, {player['clay']}, {player['obsidian']}, {player['geode']}"
        f"{rates['ore']}, {rates['clay']}, {rates['obsidian']}, {rates['geode']} "
    )

In [323]:
bp = test_bps[1]
bp

{'ore_robot': {'ore': 4},
 'clay_robot': {'ore': 2},
 'obsidian_robot': {'ore': 3, 'clay': 14},
 'geode_robot': {'ore': 2, 'obsidian': 7}}

max_geodes_bp(bp, max_time=24, toprint=False)

In [326]:
st = time.time()
print(max_geodes_bp(blueprint=input_bps[11], max_time=24, toprint=False))
print(f"Elapsed: {time.time()-st:.0f}")

11
Elapsed: 66


In [313]:
import multiprocessing as mpl

In [ ]:
def f(x): x*x

with mpl.Pool(5) as p:
    print(p.map(f, [1, 2, 3]))

In [283]:
p = default_player()

In [303]:
p = default_player()
p['ore'] = 1000
p['clay'] = 1000
p['obsidian'] = 1000
p['rate']['obsidian'] = 6
p['rate']['clay'] = 100
p['rate']['ore'] = 0
p['rate']['geode_robot'] = 10000


In [304]:
list(buildable_robots(bp, player=p))

ore 4 0
clay 14 100
obsidian 7 6


['ore_robot', 'obsidian_robot', 'geode_robot']